In [ ]:
import os
import ibis
import polars as pl

from ibis import _
from dotenv import load_dotenv

In [ ]:
load_dotenv(override=True)

In [ ]:
username = os.environ["USER"]

In [ ]:
con = ibis.postgres.connect(
    database=os.environ["DATABASE_NAME_PYTHON"],
    host=os.environ["DATABASE_HOST"],
    user=os.environ["DATABASE_USER_PYTHON"],
    password=os.environ["DATABASE_PASSWORD_PYTHON"],
    schema=os.environ["DATABASE_SCHEMA"],
)

con.list_tables()

In [ ]:
(
    con
    .table("samedwardes_terminal_weather_clean")
    .head(20)
    .to_polars()
)

In [ ]:
(
    con
    .table("vessel_history_clean")
    .mutate(
        DelayMinutes=_.ActualDepart.delta(_.ScheduledDepart, "second") / 60
    )
    .head(20)
    .to_polars()
)

In [ ]:
from great_tables import GT, loc, style
import polars.selectors as cs

In [ ]:
df = (
    con
    .table("vessel_history_clean")
    .mutate(
        DelayMinutes=_.ActualDepart.delta(_.ScheduledDepart, "second") / 60
    )
    .group_by(["Departing", "Arriving"])
    .agg(
        NumTrips=_.Departing.count(),
        AverageDelay=_.DelayMinutes.mean(),
        StandardDeviationDelay=_.DelayMinutes.std(),
    )
    .order_by(ibis.desc("AverageDelay"))
    .to_polars()
)

display(df)

(
    GT(df)
    .tab_header("Delay Stats by Route")
    .tab_spanner(label="Delay in Minutes", columns=cs.ends_with("Delay"))
    .cols_label(
        NumTrips="Number of Trips",
        AverageDelay="Average",
        StandardDeviationDelay="Standard"
    )
    .fmt_number(
        columns=cs.ends_with("Delay"),
        compact=True,
        decimals=2,
    )
    .fmt_number(
        columns=cs.ends_with("Trips"),
        compact=True,
    )
)

In [ ]:
options_list = (
    con
    .table(f"{username}_vessel_history_clean")
    .group_by(["Departing", "Arriving"])
    .aggregate(n = _.Departing.count())
    .order_by(_.n.desc())
    .to_polars()
    .to_dicts()
)

options_dict = {}

for i in options_list:
    arriving = i["Arriving"]
    departing = i["Departing"]
    n_trips = i["n"]
    value = f"{arriving} | {departing}"
    label = f"{arriving.title()} to {departing.title()} ({n_trips:,} trips)"
    options_dict[value] = label

options_dict

In [ ]:
(
    con
    .table(f"{username}_vessel_verbose_clean")
    .select("VesselName")
    .to_polars()
    .get_column("VesselName")
    .to_list()
)

In [ ]:
pl.DataFrame(con.table(f"{username}_vessel_verbose_clean").to_pandas())

In [ ]:
con.table(f"{username}_terminal_locations_clean").to_polars()

In [ ]:
con.table(f"{username}_vessel_verbose_raw").to_pandas().head()

In [ ]:
con.table(f"{username}_vessel_verbose_clean").to_pandas().head()